In [2]:
from __future__ import print_function
import os
import numpy as np
from numpy import newaxis
from sklearn import preprocessing
np.random.seed(1337)

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout
from keras.models import Model,Sequential
import sys

import pandas as pd
import random

Using TensorFlow backend.


# Reading in data

In [57]:
print("Data read started...")
data = pd.read_csv("result_tetris.csv")
data = data.as_matrix()
print ("Data read finished.")
#Transform data

print(data.shape)

Data read started...
Data read finished.
(603, 11)


# Eliminate EEG data

In [58]:
data = np.delete(data, 1, 1) #timestamp
data = np.delete(data, 2, 1) # EEG_F3_Alpha
data = np.delete(data, 3, 1) # EEG_F3_Beta
data = np.delete(data, 4, 1) # EEG_F3_Theta

data.shape

(603, 7)

In [59]:
data[1]

array(['tetris1', 84441.0, 56966.0, 1688.7, 4.378305084745762, 335.19,
       537.78], dtype=object)

# Dictionary for the levels

In [64]:
#level=["nback gyakorlas","0back","1back","2back","3back"]
level=["tetris1","tetris2","tetris3"]
level2int = dict((p, i) for i, p in enumerate(level))
int2level = dict((i, p) for i, p in enumerate(level))

# Z normalize

In [61]:
data[:, 1] = preprocessing.scale(data[:, 1])
data[:, 2] = preprocessing.scale(data[:, 2])
data[:, 3] = preprocessing.scale(data[:, 3])
data[:, 4] = preprocessing.scale(data[:, 4])
data[:, 5] = preprocessing.scale(data[:, 5])
data[:, 6] = preprocessing.scale(data[:, 6])

/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/home/amoro/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConve

In [62]:
for i in range(len(data)):
    data[i, 0] = level2int[data[i, 0]]

x_data = data[:, 1:]
y_data = data[:, 0]

# One-hot encoding

In [63]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y_one_hot = ohe.fit_transform(y_data.reshape(-1,1)).toarray()
print(y_one_hot)

[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 ..., 
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


# Shuffle data

In [65]:
indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_one_hot = y_one_hot[indices]

# Divide into train, validation and test data

In [66]:
len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_one_hot[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_one_hot[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_one_hot[end_valid:]

In [67]:
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape, x_test.shape, y_test.shape)

(422, 6) (422, 3) (90, 6) (90, 3) (91, 6) (91, 3)


# Build the net

In [70]:
model = Sequential()

model.add(Dense(200, input_shape=(6,)))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))

In [73]:
batch_size = 50
epochs = 150
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='auto')

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(x_train, y_train, nb_epoch=epochs,batch_size=batch_size, 
          callbacks=[earlyStopping], shuffle=True, validation_data = (x_valid, y_valid))

Train on 422 samples, validate on 90 samples
Epoch 1/150
422/422 [==============================] - 0s - loss: 0.4291 - acc: 0.7346 - val_loss: 0.4723 - val_acc: 0.6778
Epoch 2/150
422/422 [==============================] - 0s - loss: 0.4134 - acc: 0.7346 - val_loss: 0.4609 - val_acc: 0.7222
Epoch 3/150
422/422 [==============================] - 0s - loss: 0.3967 - acc: 0.7701 - val_loss: 0.4719 - val_acc: 0.6889
Epoch 4/150
422/422 [==============================] - 0s - loss: 0.4001 - acc: 0.7630 - val_loss: 0.4768 - val_acc: 0.7111
Epoch 5/150
422/422 [==============================] - 0s - loss: 0.4146 - acc: 0.7085 - val_loss: 0.4708 - val_acc: 0.6556
Epoch 6/150
422/422 [==============================] - 0s - loss: 0.4090 - acc: 0.7559 - val_loss: 0.4799 - val_acc: 0.6889
Epoch 7/150
422/422 [==============================] - 0s - loss: 0.4029 - acc: 0.7678 - val_loss: 0.4605 - val_acc: 0.7111
Epoch 8/150
422/422 [==============================] - 0s - loss: 0.4069 - acc: 0.7488 

In [74]:
model.evaluate(x_test, y_test, batch_size=batch_size)

91/91 [==============================] - 0s     


[0.38045224776634801, 0.78021977825479194]